# Action Menu - Bob

## Role - Responder

In this notebook we'll be going through the Action Menu Protocol. For details on the protocol, please refer to the [RFC](https://github.com/hyperledger/aries-rfcs/tree/master/features/0509-action-menu). Specifically, we'll demonstrate how Bob acts as a Responder to respond to Action Menu related requests from Alice.

A requirement for this protocol to work is an active DIDComm communication channel between Alice and Bob. To achieve that, you should have completed the [Establishing a Connection](http://localhost:8888/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial.

## 1. Initialise Bob controller

We begin by initialising the controller for Bob.

In [2]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [4]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([], defaults=True)

## 3. Check the agent has an active connection

An active connection between Alice and Bob is required. You can either:

- complete the [Establishing a Connection](http://localhost:8888/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial, or
- running the python script `create_connection.py` in the setup folder, or
- complete the did-exchange tutorial ([Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb) and [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)) 

In [6]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")

Results :  [{'my_did': 'BiqPns43A7xJboeC1GqNbu', 'created_at': '2020-10-22 03:49:47.724508Z', 'routing_state': 'none', 'accept': 'manual', 'connection_id': 'c1d1671a-d9cb-4ec8-bf68-3372c683a6bd', 'initiator': 'external', 'their_did': 'HepVkkvg1azc6PGbEkrdkD', 'invitation_mode': 'once', 'their_label': 'Alice', 'request_id': '1e8933a0-cf93-46b8-afaa-826a6ce8c58a', 'state': 'active', 'invitation_key': '2cVeFAwPbMHbtbM8A6QNyctgyLFchQtRtJ7P81QLG4ki', 'updated_at': '2020-10-22 03:49:52.638657Z'}]
Connection : {'my_did': 'BiqPns43A7xJboeC1GqNbu', 'created_at': '2020-10-22 03:49:47.724508Z', 'routing_state': 'none', 'accept': 'manual', 'connection_id': 'c1d1671a-d9cb-4ec8-bf68-3372c683a6bd', 'initiator': 'external', 'their_did': 'HepVkkvg1azc6PGbEkrdkD', 'invitation_mode': 'once', 'their_label': 'Alice', 'request_id': '1e8933a0-cf93-46b8-afaa-826a6ce8c58a', 'state': 'active', 'invitation_key': '2cVeFAwPbMHbtbM8A6QNyctgyLFchQtRtJ7P81QLG4ki', 'updated_at': '2020-10-22 03:49:52.638657Z'}
Active C

## 6. Send Action Menu to Alice

Bob responds to Alice's request by sending over the action menu that Bob supports.

In [10]:
# feel free to edit the menu options.
menu_options = [
      {
        "name": "window_prefs",
        "title": "Window Preferences",
        "description": "Window display preferences",
        "disabled": "False",
        "form": {
          "title": "Preferences",
          "description": "Window preference settings",
          "params": [{
              "name": "delay",
              "title": "Delay in seconds",
              "type": "int",
              "required": False
          }, {
              "name": "visibility",
              "title": "Visibility toggle",
              "type": "boolean",
              "required": True
          }],
          "submit-label": "Send"
        }
      }
    ]

await agent_controller.action_menu.send_menu(
    connection_id=connection_id,
    menu_description="User preferences for window settings",
    menu_errormsg="Error: item not present",
    menu_title="My Menu",
    menu_options=menu_options
)

{}

## 7. Continue with step 8 of [Alice's notebook](http://localhost:8889/notebooks/Part%208%20-%20Action%20Menu.ipynb)

## End of Tutorial

Be sure to terminate the controller before running another tutorial.

In [11]:
response = await agent_controller.terminate()
print(response)

None
